In [6]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Ashton Internship\Desktop\Chat With Excel\Operator List_processed.xlsx')

df.head()

,Region,Country,Operator,Remarks,country_code,MCC/MNC,Direct/Sponsored,handset_display_1,handset_display_2,network_tie_up,prefered_list
0,ASIA & MIDDLE EAST,Afghanistan,Afghanistan Wireless Communication (AWCC),NaN,93,41201,Direct,AF AWCC,AWCC,2G/3G/LTE,Preferred 1
1,EUROPE,Andorra,S.T.A (MOBILAND),NaN,376,21303,Direct,ANDORRA-MOBILAND,M-AND,2G/3G/LTE/VOLTE,Preferred 1
2,CENTRAL & LATIN AMERICA,Anguilla,Cable & Wireless (West Indies) Ltd. Anguilla,NaN,1-264,365840,Direct,FLOW,FLOW,2G/3G/LTE,Preferred 2
3,CENTRAL & LATIN AMERICA,Anguilla,Mossel (Jamaica) Ltd (Digicel) - NETWORK ESTEN...,NaN,1-264,338050/338880,Direct,Digicel,Digicel,2G/3G/LTE,Preferred 1
4,CENTRAL & LATIN AMERICA,Antigua and Barbuda,Digicel antigua e barbuda - NETWORK ESTENSION ...,NaN,1-268,338884,Direct,Digicel,digicel,2G/3G/LTE,Preferred 1


In [2]:
# Also, column names with spaces can be tricky and cause unexpected errors, so let us replace the spaces with underscores
# df.rename(columns={"Country code": "country_code", "Handset Display 1": "handset_display_1","Handset Display 2": "handset_display_2","Network tie-up":"network_tie_up","Prefered List":"prefered_list"}, inplace=True)
# df.head()

,Region,Country,Operator,Remarks,country_code,MCC/MNC,Direct/Sponsored,handset_display_1,handset_display_2,network_tie_up,prefered_list
0,ASIA & MIDDLE EAST,Afghanistan,Afghanistan Wireless Communication (AWCC),NaN,93,41201,Direct,AF AWCC,AWCC,2G/3G/LTE,Preferred 1
1,EUROPE,Andorra,S.T.A (MOBILAND),NaN,376,21303,Direct,ANDORRA-MOBILAND,M-AND,2G/3G/LTE/VOLTE,Preferred 1
2,CENTRAL & LATIN AMERICA,Anguilla,Cable & Wireless (West Indies) Ltd. Anguilla,NaN,1-264,365840,Direct,FLOW,FLOW,2G/3G/LTE,Preferred 2
3,CENTRAL & LATIN AMERICA,Anguilla,Mossel (Jamaica) Ltd (Digicel) - NETWORK ESTEN...,NaN,1-264,338050/338880,Direct,Digicel,Digicel,2G/3G/LTE,Preferred 1
4,CENTRAL & LATIN AMERICA,Antigua and Barbuda,Digicel antigua e barbuda - NETWORK ESTENSION ...,NaN,1-268,338884,Direct,Digicel,digicel,2G/3G/LTE,Preferred 1


In [7]:
# We can check out the length of the dataframe
len(df)

281

In [4]:
# # Save the modified DataFrame back to the Excel file
# df = df.to_excel(r'C:\Users\Ashton Internship\Desktop\Chat With Excel\Operator List_processed.xlsx', index=False)

In [8]:
import sqlite3

# Create a connection to a local database (This will create a file called mydatabase.db)
db = sqlite3.connect('test.db')

# Copy the dataframe to our SQLite database
df.to_sql('operator_list', db, if_exists='replace')

281

In [5]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///test.db")
print(db.dialect)
print(db.get_usable_table_names())

#Testing the db replies

db.run("SELECT Region FROM operator_list LIMIT 10;")

sqlite
['operator_list']


"[('ASIA & MIDDLE EAST',), ('EUROPE',), ('CENTRAL & LATIN AMERICA',), ('CENTRAL & LATIN AMERICA',), ('CENTRAL & LATIN AMERICA',), ('CENTRAL & LATIN AMERICA',), ('CENTRAL & LATIN AMERICA',), ('CENTRAL & LATIN AMERICA',), ('OCEANIA',), ('OCEANIA',)]"

In [6]:
from langchain_ollama import ChatOllama

# llm = ChatOllama(model = "deepseek-r1", temperature = 0.2,num_gpu = 1,verbose = False)
llm = ChatOllama(model = "llama3.2", temperature = 0.2,num_gpu = 1,verbose = False)
# llm = ChatOllama(model = "mistral", temperature = 0.2,num_gpu = 1,verbose = False)

In [7]:
def getDatabaseSchema():
    return db.get_table_info() if db else "Please connect to database"

In [8]:
print(getDatabaseSchema())


CREATE TABLE operator_list (
	"index" INTEGER, 
	"Region" TEXT, 
	"Country" TEXT, 
	"Operator" TEXT, 
	"Remarks" TEXT, 
	country_code TEXT, 
	"MCC/MNC" TEXT, 
	"Direct/Sponsored " TEXT, 
	handset_display_1 TEXT, 
	handset_display_2 TEXT, 
	network_tie_up TEXT, 
	prefered_list TEXT
)

/*
3 rows from operator_list table:
index	Region	Country	Operator	Remarks	country_code	MCC/MNC	Direct/Sponsored 	handset_display_1	handset_display_2	network_tie_up	prefered_list
0	ASIA & MIDDLE EAST	Afghanistan	Afghanistan Wireless Communication (AWCC)	None	93	41201	Direct	AF AWCC	AWCC	2G/3G/LTE	Preferred 1
1	EUROPE	Andorra	S.T.A (MOBILAND)	None	376	21303	Direct	ANDORRA-MOBILAND	M-AND	2G/3G/LTE/VOLTE	Preferred 1
2	CENTRAL & LATIN AMERICA	Anguilla	Cable & Wireless (West Indies) Ltd. Anguilla	None	1-264	365840	Direct	FLOW	FLOW	2G/3G/LTE	Preferred 2
*/


In [9]:
from langchain_core.prompts import ChatPromptTemplate

# template = """
#     Below is the schema of MYSQL database, read the schema carefully about the table and column names. Also take care of table or column name case sensitivity.
#     Finally answer user's question in the form of SQL query.

#     {schema}

#     please only provide the SQL query and nothing else

#     for example:
#     question: how many albums we have in database
#     SQL query: SELECT COUNT(*) FROM album
#     question: how many customers are from Brazil in the database ?
#     SQL query: SELECT COUNT(*) FROM customer WHERE country=Brazil

#     your turn :
#     question: {question}
#     SQL query :
#     please only provide the SQL query and nothing else
# """

template = """You are an expert in generating SQL queries. Below is the schema of a MySQL database. Read the schema carefully, paying attention to table names, column names, and case sensitivity.

Your task is to generate a syntactically correct SQL query that answers the user's question based on the given schema. **Only provide the SQL query and nothing else**—no explanations, no comments.

### **Schema:**  
{schema}

### **Instructions:**  
- Use **only** the provided tables and columns in the schema.  
- Be **mindful of case sensitivity** for table and column names.  
- If the user doesn't specify a limit, do **not** fetch excessive records.  
- Do **not** generate queries for non-existent tables or columns.  

### **Examples:**  
**Question:** How many albums are in the database?  
**SQL Query:** SELECT COUNT(*) FROM album;

**Question:** How many customers are from Brazil?  
**SQL Query:** SELECT COUNT(*) FROM customer WHERE country = 'Brazil';

### **Your Turn:**  
**Question:** {question}  
**SQL Query:**  
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
def runChain(question):
    chain = prompt | llm

    response = chain.invoke({
        "question": question,
        "schema": getDatabaseSchema()
    })
    return response.content

In [11]:
def getResponseForQueryResult(question, query, result):
    template2 = """You are an AI assistant skilled in SQL and natural language processing. Your task is to generate **a user-friendly response** based on a SQL query result.

Below is the schema of a MySQL database. Read the schema carefully to understand the structure, including table and column names.

### **Schema:**  
{schema}

### **Instructions:**  
- Look at the conversation history if available.  
- Read the **SQL query** and its **result set** carefully.  
- Respond in **natural, conversational language** while keeping the response **concise and informative**.  
- Maintain a friendly and professional tone.  
- If the result is empty (`[]` or `None`), respond accordingly (e.g., "There are no customers from Brazil in the database.").  

### **Examples:**  

**Question:** How many albums are in the database?  
**SQL Query:** SELECT COUNT(*) FROM album;
**Result:** `[(34,)]`  
**Response:** There are 34 albums in the database.  

**Question:** How many customers are from India?  
**SQL Query:** `ELECT COUNT(*) FROM customer WHERE country = 'India';
**Result:** `[(4,)]`  
**Response:** There are 4 customers from India in the database.  

**Question:** How many purchases were made last month?  
**SQL Query:** SELECT COUNT(*) FROM orders WHERE order_date >= '2024-02-01' AND order_date < '2024-03-01';
**Result:** `[(87,)]`  
**Response:** There were 87 purchases made last month.  

### **Your Turn:**  
**Question:** {question}  
**SQL Query:** {query}  
**Result:** {result}  
**Response:**  
    """

    prompt2 = ChatPromptTemplate.from_template(template2)
    chain2 = prompt2 | llm

    response = chain2.invoke({
        "question": question,
        "schema": getDatabaseSchema(),
        "query": query,
        "result": result
    })

    return response.content

In [12]:
question = "Name all the operators in Europe"

query = runChain(question)

print(query)

SELECT Operator FROM operator_list WHERE Region = 'EUROPE'


In [13]:
# import re

# def extract_sql_from_response(response_text):
#     # Regex pattern to match SQL queries within triple backticks
#     match = re.search(r"```sql\n(.*?)\n```", response_text, re.DOTALL)
#     if match:
#         return match.group(1).strip()  # Extract only the SQL query
#     else:
#         # If no triple backticks, try extracting based on common SQL keywords
#         match = re.search(r"(SELECT|UPDATE|DELETE|INSERT).*?;", response_text, re.DOTALL | re.IGNORECASE)
#         return match.group(0).strip() if match else None

# formatted_response = extract_sql_from_response(query)

result = db.run(query)

print(result)

[('S.T.A (MOBILAND)',), ('ONE GMBH',), ('T-Mobile + ex Telering',), ('JLLC Mobile TeleSystems (MTS)',), ('Best Belarus',), ('Belgacom Mobile (PROXIMUS)',), ('BASE NV/SA (KPN group belgium)',), ('Telenor',), ('TMobile Croazia (HTmobile)',), ('MTN Cypro (exAREEBA ex Scancom )',), ('Vodafone (Cesky Mobil -Oskar Mobil)',), ('T-Mobile Czech Republic a.s.',), ('Telefonica Czech Republic a.s.(ex Telefonica O2) - (ex Eurotel Praha)',), ('Czech republic [CZERM] T- Mobile Group',), ('HI 3G',), ('Telenor (ex Sonofon)',), ('TDC Mobil A/S',), ('Telia A/S Denmark (TELIA DK)',), ('AS EMT (EMT GSM)',), ('Radiolinja Eesti AS',), ('Tele2 Eesti AS',), ('Vodafone Faroe  (ex KALL)',), ('DNA FINNET NETWORKS (fusa conSuomen 2G Oy)',), ('Elisa corporation (ex Radiolinja Origo Oy)',), (' Telia Sonera (Sonera)',), ('Bouygues Telecom',), ('FREE MOBILE',), ('Orange France',), ('Telefonica O2 (Germany) GmbH & Co. OHG',), ('T-Mobile Deutschland GmbH (D1)',), ('COSMOTE',), ('WIND HELLAS (TELESTET)',), ('Jersey Telec

In [14]:
response = getResponseForQueryResult(question, query, result)

print(response)

Based on the provided SQL query and result, I can see that you're looking for operators in Europe. However, the result set contains a long list of operators from various countries.

To provide a more concise response, I'll extract the operators from Europe from the result set:

The following operators are available in Europe:

* S.T.A (MOBILAND) - Andorra
* ONE GMBH - Germany
* T-Mobile + ex Telering - Czech Republic
* JLLC Mobile TeleSystems (MTS) - Belarus
* Best Belarus - Belarus
* Belgacom Mobile (PROXIMUS) - Belgium
* BASE NV/SA (KPN group belgium) - Belgium
* Telenor - Norway
* TMobile Croazia (HTmobile) - Croatia
* MTN Cypro (exAREEBA ex Scancom ) - Cyprus
* Vodafone (Cesky Mobil -Oskar Mobil) - Czech Republic
* T-Mobile Czech Republic a.s. - Czech Republic
* Telefonica Czech Republic a.s.(ex Telefonica O2) - Czech Republic
* Czech republic [CZERM] T- Mobile Group - Czech Republic
* HI 3G - Switzerland
* Telenor (ex Sonofon) - Norway
* TDC Mobil A/S - Denmark
* Telia A/S Denmark